## 57단계: conv2d 함수와 pooling 함수

> .

### 57.1 im2col에 의한 전개

im2col 함수를 구현하여 아래와 같이 합성곱 계산이 편리하도록 입력 데이터를 펼쳐준다.

<img src="images/그림 57-1.png" width=600/>

이후 펼쳐진 데이터와 커널의 행렬 곱을 계산하여 출력값을 계산한다.

<img src="images/그림 57-2.png" width=600/>

### 57.2 conv2d 함수 구현

이 책에서는 im2col 함수를 블랙박스처럼 사용한다.

함수의 각 파라미터는 아래 표를 참고한다.

<img src="images/표 57-1.png" width=500/>

kernel_size가 int면 높이와 너비를 동일하게, (int, int)면 (높이, 너비)로 해석한다. (stride, pad도 동일)

In [1]:
import numpy as np
import dezero.functions as F

x1 = np.random.rand(1, 3, 7, 7)  # 배치 크기 = 1
col1 = F.im2col(x1, kernel_size=5, stride=1, pad=0, to_matrix=True)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)  # 배치 크기 = 10
kernel_size = (5, 5)
stride = (1, 1)
pad = (0, 0)
col2 = F.im2col(x2, kernel_size, stride, pad, to_matrix=True)
print(col2.shape)

(9, 75)
(90, 75)


im2col 함수를 사용한 합성곱 연산 함수를 구현하기 전, `pair`라는 편의 함수를 구현한다.

In [2]:
# dezero/utils.py
def pair(x):
    if isinstance(x, int):
        return (x, x)
    elif isinstance(x, tuple):
        assert len(x) == 2
        return x
    else:
        raise ValueError

In [3]:
from dezero.utils import pair

print(pair(1))
print(pair((1, 2)))

(1, 1)
(1, 2)


이제 합성곱 연산을 수행하는 함수 `conv2d_simple`을 구현한다.

In [4]:
# dezero/functions_conv.py
from dezero.core import as_variable
from dezero.functions import im2col, linear
from dezero.utils import pair, get_conv_outsize

def conv2d_simple(x, W, b=None, stride=1, pad=0):
    x, W = as_variable(x), as_variable(W)

    Weight = W
    N, C, H, W = x.shape
    OC, C, KH, KW = Weight.shape
    SH, SW = pair(stride)
    PH, PW = pair(pad)
    OH = get_conv_outsize(H, KH, SH, PH)
    OW = get_conv_outsize(W, KW, SW, PW)

    col = im2col(x, (KH, KW), stride, pad, to_matrix=True)  # 입력 데이터 펼치기
    Weight = Weight.reshape(OC, -1).transpose()  # 커널 Weight 펼친 후 transpose
    t = linear(col, Weight, b)  # 행렬 곱 계산
    y = t.reshape(N, OH, OW, OC).transpose(0, 3, 1, 2)  # 출력 shape 조정
    return y

<img src="images/그림 57-3.png" width=500/>

In [5]:
from dezero import Variable

N, C, H, W = 1, 5, 15, 15
OC, (KH, KW) = 8, (3, 3)

x = Variable(np.random.randn(N, C, H, W))
W = np.random.randn(OC, C, KH, KW)
y = F.conv2d_simple(x, W, b=None, stride=1, pad=1)
y.backward()

print(y.shape)
print(x.grad.shape)

(1, 8, 15, 15)
(1, 5, 15, 15)


Function 클래스 상속한 버전은 dezero/functions_conv.py 참고

### 57.3 Conv2d 계층 구현

In [6]:
# dezero/layers.py
from dezero import Layer, Parameter, cuda

class Conv2d(Layer):
    def __init__(self, out_channels, kernel_size, stride=1,
                 pad=0, nobias=False, dtype=np.float32, in_channels=None):

        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.pad = pad
        self.dtype = dtype

        self.W = Parameter(None, name='W')
        if in_channels is not None:
            self._init_W()

        if nobias:
            self.b = None
        else:
            self.b = Parameter(np.zeros(out_channels, dtype=dtype), name='b')

    def _init_W(self, xp=np):
        C, OC = self.in_channels, self.out_channels
        KH, KW = pair(self.kernel_size)
        scale = np.sqrt(1 / (C * KH * KW))
        W_data = xp.random.randn(OC, C, KH, KW).astype(self.dtype) * scale
        self.W.data = W_data

    def forward(self, x):
        if self.W.data is None:
            self.in_channels = x.shape[1]
            xp = cuda.get_array_module(x)
            self._init_W(xp)

        y = F.conv2d(x, self.W, self.b, self.stride, self.pad)
        return y

<img src="images/표 57-2.png" width=500/>

### 57.4 pooling 함수 구현

<img src="images/그림 57-4.png" width=600/>
<br/>
<img src="images/그림 57-5.png" width=600/>

In [7]:
# dezero/functions_conv.py

def pooling_simple(x, kernel_size, stride=1, pad=0):
    x = as_variable(x)

    N, C, H, W = x.shape
    KH, KW = pair(kernel_size)
    PH, PW = pair(pad)
    SH, SW = pair(stride)
    OH = get_conv_outsize(H, KH, SH, PH)
    OW = get_conv_outsize(W, KW, SW, PW)

    col = im2col(x, kernel_size, stride, pad, to_matrix=True)  # 입력 데이터 펼치기
    col = col.reshape(-1, KH * KW)
    y = col.max(axis=1)  # 각 행의 최댓값 찾기
    y = y.reshape(N, OH, OW, C).transpose(0, 3, 1, 2)  # shape 변환
    return y